# Figuring out the html tags etc

In [102]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
import string

In [24]:
URL="https://www.meesho.com/dresses-women/pl/3j3"

In [25]:
HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

In [26]:
webpage=requests.get(URL,headers=HEADERS)

In [27]:
webpage

<Response [200]>

In [28]:
soup= BeautifulSoup(webpage.content,parser="html.parser")

In [49]:
dresses=soup.find_all('div',class_=re.compile("sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA")) 
for dress in dresses:
    links= dress.find('a').get('href')
    price=dress.find('h5',class_=re.compile("sc-eDvSVe")).text
    #name=dress.find('p').text
    print(links)
    print(price)
    #print(name)
    print(" ")

/magenta-flower-printed-rayon-dungaree-dress/p/6m09z1
₹620
Classic Latest Women Dresses
 
/pretty-modern-women-dresses/p/3lp8s9
₹191
Urbane Ravishing Women Dresses
 
/aj19es49a/p/15wrv3
₹208
Pretty Modern Women Dresses
 
/women-stylish-and-trendy-dress/p/54d5e9
₹316
Comfy Latest Women Dresses
 
/puff-sleeve-stylist-floral-printed-dresses-for-women/p/6d7ix3
₹363
Stylish Fashionista Women Dresses
 
/western-dress-for-girls-stylish-dresses-long-gown-indo-women-frocks-cotton-maxi-with-frok-top-short-age-ladies-combo-dupatta-woman-tops-formal-simple-one-piece-frock-knee-length-full-sleeve-party-wear-white-womens-kurti-pink-pleated-trendy-dungaree-shirt-tshirt-1piece-13-years-crop-black-new/p/3cpbab
₹312
Pretty Partywear Women Dresses
 
/party-wear-for-girls/p/6pp7wr
₹204
Classic Graceful Women Dresses
 
/honky-tonky-womens-animal-round-neck-pink-dresses/p/6l6vs6
₹349
Honky Tonky Women's Dresses
 
/designer-dress/p/62nqlx
₹302
Comfy Sensational Women Dresses
 
/new-fashion-western-dress-for-

In [56]:
dress=dresses[0]
link=dress.find('a').get("href")

In [57]:
product="https://www.meesho.com"+link

In [58]:
product

'https://www.meesho.com/magenta-flower-printed-rayon-dungaree-dress/p/6m09z1'

In [60]:
new_webpage=requests.get(product,headers=HEADERS)

In [61]:
new_webpage

<Response [200]>

In [62]:
new_soup= BeautifulSoup(new_webpage.content,parser="html.parser")

In [64]:
name=new_soup.find('title').text
print(name)

Magenta Flower Printed Rayon Dungaree Dress


In [67]:
rating=new_soup.find('h1',class_=re.compile('sc-eDvSVe')).text
print(rating)

4.1


# Making it functional

In [111]:

def get_title(soup):

    
        # Outer Tag Object
        title = soup.find('title').text
        # Title as a string value
   
        return title

# Function to extract Product Price
def get_price(soup):

        price = soup.find("h4", class_=re.compile("sc-eDvSVe")).text
        price=re.sub('\W+','',price)
        return price

# Function to extract Product Rating
def get_rating(soup):

        try:
            rating = soup.find('h1',class_=re.compile('sc-eDvSVe')).text
        except:
            rating = ""

        return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    
        review_count = soup.find("span", class_=re.compile('sc-eDvSVe XndEO')).text
        review_count=re.sub('\W+','', review_count)
        return ''.join(i for i in review_count if i.isdigit())
    

# Function to extract Availability Status
def seller(soup):
    
        seller = soup.find("span", class_=re.compile('sc-eDvSVe dpdDrR ShopCardstyled__ShopName-sc-du9pku-6 bdcHGu ShopCardstyled__ShopName-sc-du9pku-6 ')).text
        return seller
def get_discount(soup):
    try:
        
        discount=soup.find("span",class_=re.compile('sc-eDvSVe dOqdSt')).text
        discount=re.sub('\W+','', discount)
        

    except AttributeError:
        discount=''
    return ''.join(i for i in discount if i.isdigit())


In [117]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.meesho.com/dresses-women/pl/3j3"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links_list = []
    #container=soup.find_all('div',class_='sc-bcXHqe ProductList__PLPContainer-sc-8lnc8o-1 cppHWG hSZegP')
    dresses=soup.find_all('div',class_=re.compile("sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0")) 
    for dress in dresses:
             links= dress.find('a').get('href')
     
             links_list.append(links)

    print(links_list)
    d = {"title":[], "price":[], "rating":[], "reviews":[],"seller":[],"discount":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.meesho.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['seller'].append(seller(new_soup))
        d['discount'].append(get_discount(new_soup))
    meesho_df = pd.DataFrame.from_dict(d)
    meesho_df['title'].replace('', np.nan, inplace=True)
    meesho_df = meesho_df.dropna(subset=['title'])
    meesho_df.to_csv("meesho_data.csv", header=True, index=False)

['/classy-elegant-white-puffed-sleeve-skater-dress/p/6yqv3s', '/aj19es49a/p/15wrv3', '/women-stylish-and-trendy-dress/p/54d5e9', '/party-wear-for-girls/p/6pp7wr', '/trendy-graceful-women-dresses/p/4z2zsz', '/designer-dress/p/62nqlx', '/honky-tonky-womens-animal-round-neck-pink-dresses/p/6l6vs6', '/women-dresses/p/53jdju', '/graceful-v-neck-full-sleeve-floral-print-gown-elegant-evening-dress/p/5utoh7', '/white-short-delta-frock/p/66mmur', '/tiger-print-dress-trending/p/27bwiy', '/women-beautiful-short-dress-black/p/6iqp99', '/new-stylish-bollywood-nora-style-dress/p/37z3nq', '/rkk-collections-womens-girls-a-line-dress-casual-wear-printed-short-sleeve-dress-fabric-cotton-in-color-neon/p/5ndmg2', '/trendy-graceful-v-neck-full-sleeve-gown-elegant-evening-dress/p/5utohd', '/new-stylish-bodycon-lavender-purple-dress-for-womens-black-midi-knee-length-dress-party-brithday-special-dress-new-new-part-wear-dress/p/6kay41', '/fancy-latest-women-dresses/p/213ycp', '/women-knee-lenth-dress-women-cas

In [113]:
meesho_df

,title,price,rating,reviews,seller,discount
0,Women knee lenth Dress | Women casual Dress's,350,3.8,77,Women tshirt,7
1,Pretty Modern Women Dresses,233,3.8,2996,EVERGREEN COLLECTION,8
2,AJ19ES49A,208,3.9,97884,AJAY CRE#,14
3,WOMEN STYLISH AND TRENDY DRESS,316,4.1,26106,GT-GARMENTS,2
4,Puff Sleeve Stylist Floral Printed Dresses For...,363,4.1,4440,BHAGAT_FASHION,11
5,Women Gown | Formal Gown | A-Line Gown for Women,354,3.7,405,Women tshirt,7
6,Party Wear For Girls,204,4.2,2507,The Quality Clothes,8
7,Designer dress,302,4.1,7407,MANOJAVA,6
8,Honky Tonky Women's Animal Round Neck Pink Dre...,349,4.2,7199,Honky Tonky,83
9,New Special Pretty Dress,266,3.8,22787,A D S CREATION,8


#### Top five dresses by rating /reviews

In [118]:
top_rated = meesho_df.sort_values(by='rating', ascending=False).head(5)
top_rated[['title', 'price', 'rating', 'reviews', 'seller', 'discount']]


,title,price,rating,reviews,seller,discount
11,Women Beautiful Short Dress (Black),263,4.3,7879,VARTIKA ENTERPRISES,8
12,NEW STYLISH BOLLYWOOD NORA STYLE DRESS,293,4.3,5406,KI STUDIO,5
3,Party Wear For Girls,204,4.2,2507,The Quality Clothes,8
6,Honky Tonky Women's Animal Round Neck Pink Dre...,349,4.2,7199,Honky Tonky,83
15,new Stylish Bodycon lavender purple dress for ...,416,4.2,2996,feyola enterparise,11


In [119]:
most_reviewed = meesho_df.sort_values(by='reviews', ascending=False).head(5)
most_reviewed[['title', 'price', 'rating', 'reviews', 'seller', 'discount']]


,title,price,rating,reviews,seller,discount
1,AJ19ES49A,208,3.9,97884,AJAY CRE#,14
11,Women Beautiful Short Dress (Black),263,4.3,7879,VARTIKA ENTERPRISES,8
17,Women knee lenth Dress | Women casual Dress's,350,3.8,77,Women tshirt,7
5,Designer dress,302,4.1,7407,MANOJAVA,6
6,Honky Tonky Women's Animal Round Neck Pink Dre...,349,4.2,7199,Honky Tonky,83
